In [2]:
import alpaca_trade_api as tradeapi
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
# Define your keys
import alpaca_trade_api as tradeapi
# API Info for fetching data, portfolio, etc. from Alpaca
BASE_URL = "https://app.alpaca.markets/paper/dashboard/overview"
ALPACA_API_KEY = "<PKL2ZVK5ZQXEV9WS80SJ>"
ALPACA_SECRET_KEY = "<p5dyc2NGGO0rgmo3QRAjb6hzd5L9nR0lfSafZKQc>"
# Instantiate REST API Connection
api = tradeapi.REST(key_id=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY,
                    base_url=BASE_URL, api_version='v2')
# Define the date range (last 4 months)
end_date = datetime.now()
start_date = end_date - timedelta(days=4*30)
# Fetch daily data for Apple stock
symbol = 'AAPL'
stock_data = yf.download(symbol, start=start_date, end=end_date, interval='1h')
# Existing code for calculating pivot points, support/resistance, etc.
pivot_point = (stock_data['High'] + stock_data['Low'] + stock_data['Close']) / 3
# Calculate support and resistance levels
first_support = (2 * pivot_point) - stock_data['High']
first_resistance = (2 * pivot_point) - stock_data['Low']
second_support = pivot_point - (stock_data['High'] - stock_data['Low'])
second_resistance = pivot_point + (stock_data['High'] - stock_data['Low'])
third_support = pivot_point - 2 * (stock_data['High'] - stock_data['Low'])
third_resistance = pivot_point + 2 * (stock_data['High'] - stock_data['Low'])
# Create a new DataFrame to store the pivot points and support/resistance levels
pivot_df = pd.DataFrame(index=stock_data.index)
pivot_df['Pivot'] = pivot_point
pivot_df['Support 1'] = first_support
pivot_df['Resistance 1'] = first_resistance
pivot_df['Support 2'] = second_support
pivot_df['Resistance 2'] = second_resistance
pivot_df['Support 3'] = third_support
pivot_df['Resistance 3'] = third_resistance
# Merge the pivot DataFrame with the stock data DataFrame
stock_data_with_pivot = pd.concat([stock_data, pivot_df], axis=1)
# Create signals based on pivot points
stock_data_with_pivot['Signal'] = 0
stock_data_with_pivot.loc[stock_data_with_pivot['Low'] < stock_data_with_pivot['Support 1'], 'Signal'] = 1
stock_data_with_pivot.loc[stock_data_with_pivot['Low'] < stock_data_with_pivot['Support 2'], 'Signal'] = 2
stock_data_with_pivot.loc[stock_data_with_pivot['Low'] < stock_data_with_pivot['Support 3'], 'Signal'] = 3
stock_data_with_pivot.loc[stock_data_with_pivot['High'] > stock_data_with_pivot['Resistance 1'], 'Signal'] = -1
stock_data_with_pivot.loc[stock_data_with_pivot['High'] > stock_data_with_pivot['Resistance 2'], 'Signal'] = -2
stock_data_with_pivot.loc[stock_data_with_pivot['High'] > stock_data_with_pivot['Resistance 3'], 'Signal'] = -3
# Generate trading orders based on the signals
from alpaca_trade_api.rest import APIError
for index, row in stock_data_with_pivot.iterrows():
    try:
        if row['Signal'] > 0:
            api.submit_order(
                symbol=symbol,
                qty='100',
                side='buy',
                type='market',
                time_in_force='gtc'
            )
        elif row['Signal'] < 0:
            # Get the current position to ensure we don't short sell
            position = api.get_position(symbol)
            if position.qty >= 100:  # Adjust this value as needed
                api.submit_order(
                    symbol=symbol,
                    qty='100',
                    side='sell',
                    type='market',
                    time_in_force='gtc'
                )
    except APIError as e:
        print(f'Error when processing order for {symbol}: {str(e)}')

[*********************100%***********************]  1 of 1 completed


HTTPError: 405 Client Error: Not Allowed for url: https://app.alpaca.markets/paper/dashboard/overview/v2/orders